In [ ]:
import numpy as np
import pandas as pd
import math
import sklearn.preprocessing
import datetime
from TimeBasedCV import TimeBasedCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import make_scorer, r2_score
import pickle
from sklearn.linear_model import HuberRegressor
pd.set_option('display.max_rows', None)
# more
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [ ]:
# df = pd.read_csv('data/factors_1965.csv', parse_dates=['DATE'])

In [ ]:
# with open('data/features_1965.pkl', 'wb') as f:
#     pickle.dump(df, f)

with open('data/features_1965.pkl', 'rb') as f:
    df = pickle.load(f)
    print(df.head())

In [ ]:
#Sort observations by date and stock id
df[df.columns[2:]] = df[df.columns[2:]].astype('float32')
df = df.sort_values(by = ['DATE', 'permno'], ascending = True)
df.head()


In [ ]:

df['permno2'] = df['permno'].copy()
df['DATE2'] = df['DATE'].copy()
df = df.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
df['mvel12'] = df['mvel1'].copy()

In [ ]:
p=0.3 
df_large= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)  
df_small = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1')).reset_index(drop=True)  


In [ ]:
#Standardize all independent variables

features = df.columns[~df.columns.isin(['DATE', 'DATE2', "mvel2",'sic2' ,'permno',"permno2",'risk_premium'])].tolist()
df[features]=df.groupby('DATE')[features].rank(pct=True)

df[features] = 2*df[features] - 1

In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


X = df[['DATE', 'macro_mkt-rf', 'macro_tbl', 'mvel1', 'bm']]
y = df[['risk_premium']]

#Empty containers to save results from each window

predictions = []
y_test_list =[]
dates = []
dic_r2_all = {}


for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(1981,1,31), second_split_date= datetime.date(1991,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]

    lm = LinearRegression()
   
    lm.fit(np.concatenate((X_train, X_val)), np.concatenate((y_train, y_val)))
    #Use test set to generate final predictions 
    preds = lm.predict(X_test)

    #Save predictions, dates and the true values of the dependent variable to list  
    predictions.append(preds)
    dates.append(y_test.index)
    y_test_list.append(y_test)

    r2 = 1-np.sum(pow(y_test-preds,2))/np.sum(pow(y_test,2))

    #Save OOS model performance and the respective month to dictionary
    dic_r2_all["r2." + str(y_test.index)] = r2

predictions_all= np.concatenate(predictions, axis=0)
y_test_list_all= np.concatenate(y_test_list, axis=0) 
dates_all= np.concatenate(dates, axis=0)

#Calculate OOS model performance over the entire test period in line with Gu et al (2020)
R2OOS_LR = 1-np.sum(pow(y_test_list_all-predictions_all,2))/np.sum(pow(y_test_list_all,2))
print("R2OOS Linear Regression Full: ", R2OOS_LR)

In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


X = df_large[['DATE', 'macro_mkt-rf', 'macro_tbl', 'mvel1', 'bm']]
y = df_large[['risk_premium']]

#Empty containers to save results from each window

predictions_top = []
y_test_list_top =[]
dates_top = []
dic_r2_all_top = {}

for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(1981,1,31), second_split_date= datetime.date(1991,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]
    
    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]
    

    lm = LinearRegression()
    lm.fit(np.concatenate((X_train, X_val)), np.concatenate((y_train, y_val)))
    preds = lm.predict(X_test) 
    predictions_top.append(preds)
    dates_top.append(y_test.index)
    y_test_list_top.append(y_test)
    
    r2_top = 1-np.sum(pow(y_test-preds,2))/np.sum(pow(y_test,2))
    dic_r2_all_top["r2." + str(y_test.index)] = r2

predictions_all_top= np.concatenate(predictions_top, axis=0)
y_test_list_all_top= np.concatenate(y_test_list_top, axis=0) 
dates_all_top= np.concatenate(dates_top, axis=0)

R2OOS_LR_top = 1-np.sum(pow(y_test_list_all_top-predictions_all_top,2))/np.sum(pow(y_test_list_all_top,2))
print("R2OOS Linear Regression Top: ", R2OOS_LR_top)

In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


X = df_small[['DATE', 'macro_mkt-rf', 'macro_tbl', 'mvel1', 'bm']]
y = df_small[['risk_premium']]

#Empty containers to save results from each window

predictions_bottom = []
y_test_list_bottom =[]
dates_bottom = []
dic_r2_all_bottom = {}


for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(1981,1,31), second_split_date= datetime.date(1991,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]
    
    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]
    

    lm = LinearRegression()
    lm.fit(np.concatenate((X_train, X_val)), np.concatenate((y_train, y_val)))
    preds = lm.predict(X_test) 
    predictions_bottom.append(preds)
    dates_bottom.append(y_test.index)
    y_test_list_bottom.append(y_test)
    
    r2_bottom = 1-np.sum(pow(y_test-preds,2))/np.sum(pow(y_test,2))
    dic_r2_all_bottom["r2." + str(y_test.index)] = r2

predictions_all_bottom= np.concatenate(predictions_bottom, axis=0)
y_test_list_all_bottom= np.concatenate(y_test_list_bottom, axis=0) 
dates_all_bottom= np.concatenate(dates_bottom, axis=0)

R2OOS_LR_bottom = 1-np.sum(pow(y_test_list_all_bottom-predictions_all_bottom,2))/np.sum(pow(y_test_list_all_bottom,2))
print("R2OOS Linear Regression Bottom: ", R2OOS_LR_bottom)

In [ ]:

chart = np.array([[R2OOS_LR],
                  [R2OOS_LR_top],
                  [R2OOS_LR_bottom]])

r2_lm = pd.DataFrame(chart, columns=['Linear Regression (Fama-French)'],
                     index=['Full Sample', 'Large Firms', 'Small Firms'])

r2_lm

In [ ]:
r2_lm.to_csv(r'r2_linear_model_FF.csv')

In [ ]:
yhat = predictions_all.tolist()
y_true = y_test_list_all.tolist()
i = dates_all.tolist()

results = pd.DataFrame(
    {'identifier': i,
     'yhat': yhat,
     'y_true': y_true
    })

results["identifier"]= results["identifier"].astype("str")
results["date"] = results["identifier"].str[12:22]
results["id"] = results["identifier"].str[36:51]
results.drop(["identifier"],axis = 1, inplace=True)
results['date'] = pd.to_datetime(results['date'], format='%Y-%m-%d')
results['MonthYear'] = results['date'].dt.to_period('M')
results = results.sort_values(by = ['date', 'id'], ascending = True)
results = results.set_index(['MonthYear','id'])
results

In [ ]:
lm = results.reset_index()
lm.to_csv('predictions/lm_ff.csv', index=False)